# Naive CAPM Calculations #

### CAPM Model using Simple Linear Regression Model ###

In [1]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Plots
import matplotlib.pyplot as plt

# Handle Files
import sys
import os

# Import Local Functions
sys.path.append(os.path.abspath("../source"))
from functions import import_daily_financial_data
from functions import get_fred_data
from capm_toolkit import annual_to_daily_rate

In [2]:
# Stock Data
ticker = 'NVDA'

df_stock = import_daily_financial_data(
        ticker=ticker, 
        start_date='2015-01-01', 
        end_date='2025-01-01', 
        returns=True
    )

df_stock

In [3]:
# FRED key
key = '0174cb93931388a2bf305663e4117fd3'

In [4]:
# We will use the 10-year bond rate as the RFR
rf = get_fred_data('DGS10', fred_key=key)
rf.index = pd.to_datetime(rf.index)
rf = rf.loc['2015-01-01':]
rf.name = 'risk_free_rate'
rf.dropna(inplace=True)

# Daily Risk Free Rate
daily_rfr = annual_to_daily_rate(rf)

daily_rfr

In [5]:
# Benchmark Data
data_sp500 = import_daily_financial_data(
    ticker='^GSPC', 
    start_date='2015-01-01', 
    end_date='2025-01-01', 
    returns=True
)

data_sp500

In [6]:
# Create the DataFrame
data = pd.DataFrame()

data['stock'] = df_stock['returns']
data['daily_rfr'] = daily_rfr
data['benchmark'] = data_sp500['returns']

data = data.dropna()

data

In [7]:
# We use the excess returns
data['stock_excess'] = data['stock'] - data['daily_rfr']
data['market_excess'] = data['benchmark'] - data['daily_rfr']
data.dropna(inplace = True)

data

In [8]:
# We know how to calculate a beta, but not a rolling beta
data['rolling_cov'] = data['stock_excess'].rolling(window=252).cov(data['market_excess'])
data['rolling_var_market'] = data['market_excess'].rolling(window=252).var()

# Rolling beta
data['rolling_beta'] = data['rolling_cov'] / data['rolling_var_market']

data.dropna()

In [9]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(data['rolling_beta'], label='Beta', color='blue', alpha=0.7)
plt.axhline(y=data['rolling_beta'].mean(), color='black', linestyle='dashed', label='Beta Mean')

# Config
plt.title('Beta Time Series')
plt.xlabel('Time')
plt.ylabel('Beta')
plt.legend()

# Show
plt.show()

In [10]:
# Now we are interested in getting the alpha
data['rolling_avg_stock'] = data['stock_excess'].rolling(window=252).mean()
data['rolling_avg_market'] = data['market_excess'].rolling(window=252).mean()

data['rolling_alpha'] = data['rolling_avg_stock'] - data['rolling_beta'] * data['rolling_avg_market']

data.dropna()

In [11]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(data['rolling_alpha'], label='Alpha', color='orange', alpha=0.7)
plt.axhline(y=data['rolling_alpha'].mean(), color='black', linestyle='dashed', label='Alpha Mean')

# Config
plt.title('Alpha Time Series')
plt.xlabel('Time')
plt.ylabel('Alpha')
plt.legend()

# Show
plt.show()

In [12]:
# Create Figure
fig, ax1 = plt.subplots(dpi = 300)

# Rolling Beta
data['rolling_beta'].plot(color = 'blue', ax = ax1, alpha=0.7)
ax1.set_xlabel('Date')
ax1.set_ylabel(
    'Beta', 
    color='blue'
    )

# Rolling Alpha
ax2 = ax1.twinx()

data['rolling_alpha'].plot(color = 'orange', ax = ax2, alpha=0.7)
ax2.set_ylabel(
    'Alpha', 
    color='orange'
    )

plt.title('Beta and Alpha Time Series')
plt.show()

In [13]:
# Security Market Line
# Parameters
risk_free_rate = data['daily_rfr'].iloc[-1]
market_risk_premium = data['rolling_avg_market'].iloc[-1]

# Random Betas
betas = np.linspace(0, 3, 100)  # RANGE

# Expected Returns
expected_returns = risk_free_rate + market_risk_premium * betas

# Plot
plt.figure(figsize=(10, 6))
plt.plot(betas, expected_returns, color='black', linestyle='--', label='Security Market Line')
plt.scatter(data['rolling_beta'].iloc[-1], data['rolling_avg_stock'].iloc[-1], color='red', s=50, label='expected returns')  

# Config
plt.title('Security Market Line (SML)')
plt.xlabel('Beta')
plt.ylabel('Expected Return')

# RFR Line
plt.axhline(y=risk_free_rate, color='grey', linestyle='--', label='Risk-Free Rate')

# Legends
plt.legend()

plt.grid(True)
plt.show()

In [14]:
# Now we have to calculate the Treynor's Ratio
data['treynors_ratio'] = data['stock_excess'].mul(100).div(data['rolling_beta'])

data.dropna()

In [15]:
# And the Sharpe Ratio
data['rolling_std_stock'] = data['stock'].rolling(window=252).std()
data['sharpe_ratio'] = data['stock_excess'].div(data['rolling_std_stock'])

data.dropna()

In [16]:
# Create Plot to demonstrate the Sharpe Ratio is always smaller than the Treynors Ratio

plt.figure(figsize=(10, 6))
plt.plot(abs(data['treynors_ratio']), label='Treynors Ratio', color='green', alpha=0.7)
plt.plot(abs(data['sharpe_ratio']), label='Sharpe Ratio', color='red', alpha=0.7)

# Config
plt.title('Ratios Time Series')
plt.xlabel('Time')
plt.ylabel('Values')
plt.legend()

# Show
plt.show()

In [17]:
plt.figure(figsize=(10, 6))
plt.plot(abs(data['treynors_ratio'] / abs(data['sharpe_ratio'])), label='Ratios Difference', color='green', alpha=0.7)
plt.axhline(y=1, color='grey', linestyle='--')

# Config
plt.title('Ratios Time Series')
plt.xlabel('Time')
plt.ylabel('Values')
plt.legend()

# Show
plt.show()

In [23]:
daily_rfr.index.name = 'Date'
daily_rfr.to_csv(r'..\additional_data\rfr.csv', index=True)